In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import sequence

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Xử lí csv

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/DoAnLTAT2/DuLieu/train.txt',error_bad_lines=False)
df.to_csv('/content/drive/MyDrive/DoAnLTAT2/DuLieu/train.csv')

ds = pd.read_csv('/content/drive/MyDrive/DoAnLTAT2/DuLieu/train.csv')
print(ds)

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/DoAnLTAT2/DuLieu/test.txt',error_bad_lines=False)
df.to_csv('/content/drive/MyDrive/DoAnLTAT2/DuLieu/test.csv')

ds = pd.read_csv('/content/drive/MyDrive/DoAnLTAT2/DuLieu/test.csv')
print(ds)

# Xử lí pre-training

# GetData & LoadData

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Nov 11 14:51:39 2017

This file implements a LSTM network that is capable of leveraging the historical vulnerable function data for learning the representations.

"""
import tensorflow as tf
import time
import pickle
import csv
import numpy as np

from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding, Bidirectional
from keras.layers import LSTM
from keras.layers import GlobalMaxPooling1D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.callbacks import TensorBoard, CSVLogger
from keras import backend as K
import pandas as pd

script_start_time = time.time()

print ("Script starts at: " + str(script_start_time))

# ------------------------------------------------------------ #
# Parameters used
MAX_LEN = 1000 # The Padding Length for each sample.
EMBEDDING_DIM = 100 # The Embedding Dimension for each element within the sequence of a data sample. 

NUM_TRAIN_SAMPLE = 19326
NUM_VALIDATION_SAMPLE = 8283
NUM_TEST_SAMPLE = 4921
BATCH_SIZE = 32
EPOCHS = 50

#working_dir = '/home/your/user/name/TransferRepresentationLearning/ffmpeg/'

#w2v_dir = '/content/drive/MyDrive/DoAnLTAT2/w2c/'

log_path = '/content/drive/MyDrive/DoAnLTAT2/Model/'

# The path where the trained models are saved.
model_saved_path = '/content/drive/MyDrive/DoAnLTAT2/Model_save_path/'

saved_model_name = "1st_1000_100_32_90_test_on_ffmpeg"


# Use for load module
def LoadSavedData(path):
    with open(path, 'rb') as f:
        loaded_data = pickle.load(f)
    return loaded_data

# Get the csv file and convert it to a list.
def getData(filePath):
    df = pd.read_csv(filePath, sep=",")
    df_id = df['Unnamed: 0']
    print(df_id)
    df_value = df.drop(['Unnamed: 0'], axis=1)
    df_list = df_value.values.tolist()
    df_id_list = df_id.values.tolist()
    temp = []
    ####
    for i in df_list:
        # Get rid of 'NaN' values.
        i = [x for x in i if str(x) != 'nan']
        temp.append(i)

    return temp, df_id_list
  
def getData2(filePath):
    df = pd.read_csv(filePath, sep=",")
    df_list = df.values.tolist()
    temp = []
    for i in df_list:
        # Get rid of 'NaN' values.
        i = [x for x in i if str(x) != 'nan']
        temp.append(i)
    return temp

def generateLabels(input_arr):
    temp_arr = []
    for func_id in input_arr:
        temp_sub_arr = []
        if "1:::" or "cve" in func_id:
            temp_sub_arr.append(1)
        else:
            temp_sub_arr.append(0)
        temp_arr.append(temp_sub_arr)
    return np.asarray(temp_arr)


def storeOuput(arr, path):
    with open(path, 'w') as myfile:
        wr = csv.writer(myfile)
        wr.writerow(arr)

# get training data
#training_list,training_list_id = getData('/content/drive/MyDrive/DoAnLTAT2/DuLieu/train.csv')


training_list = getData2('/content/drive/MyDrive/DoAnLTAT2/Data/Code Metric/FFmpeg/train_ffmpeg_cm.csv')
training_list_id = getData2('/content/drive/MyDrive/DoAnLTAT2/Data/Code Metric/FFmpeg/train_ffmpeg_id.csv')


testing_list =  getData2('/content/drive/MyDrive/DoAnLTAT2/Data/Code Metric/FFmpeg/test_ffmpeg_cm.csv')
testing_list_id = getData2('/content/drive/MyDrive/DoAnLTAT2/Data/Code Metric/FFmpeg/test_ffmpeg_id.csv')


print ("The number of training functions: " + str(len(training_list)) + "  ID: " + str(len(training_list_id)))
print ("The number of testing functions: " + str(len(testing_list)) + "  ID: " + str(len(testing_list_id)))



Script starts at: 1672201074.484473
The number of training functions: 1229  ID: 1229
The number of testing functions: 3690  ID: 3690


# Word2vec: ánh xạ thành các token 

In [ ]:
def JoinSubLists(list_to_join):
    new_list = []
    
    for sub_list_token in list_to_join:
        new_line = ','.join(sub_list_token)
        new_list.append(new_line)
    return new_list

new_training_list = JoinSubLists(training_list)
new_testing_list = JoinSubLists(testing_list)



['2,2,6', '1,7', '2,2,6,.,1', '3,3', '1,8,8', '0', '0,.,1', '1,1', '2,8,0', '2', '1,9,7', '2,2,8', '3,2', '0,.,2', '1,9,6', '3,4', '3,4,.,1', '4,0', '1', '1,5', '0,.,3', '5', '0,.,4']


# và embedding thành các high presentation token sequence

In [ ]:
# Da set max len o tren
print("max_len " + str(MAX_LEN))
print('Pad sequences (samples x time)')


train_sequences_pad = sequence.pad_sequences(training_list, maxlen=MAX_LEN, padding='post')
test_sequences_pad = sequence.pad_sequences(testing_list, maxlen=MAX_LEN, padding='post')



print(train_sequences_pad.shape)
print(train_sequences_pad)


# ------------------------------------------------------------ #
# 3. Split the data into 3 parts: training, validation and testing.

# Use the id as the 'test' set because the id contains the 'cve' keyword which can be used as the label later.
# When the samples in the train_set_x are
train_set_x, test_validation_set_x, train_set_y_id, test_validation_set_y = train_test_split(train_sequences_pad, training_list_id,
                                                                                             test_size=0.35,
                                                                                             random_state=43)

print("The training set: ")
print(train_set_x)

# print test_validation_set_x

print("The length of training set: " + str(len(train_set_x)) + ". The length of training set id list: " + str(
    len(train_set_y_id)))

# Further split the data set into two parts, to form training, validation and testing parts with the ratio of 0.65:0.2:0.15
validation_set_x, test_set_x, validation_set_id, test_set_id = train_test_split(test_validation_set_x,
                                                                                test_validation_set_y, test_size=0.43,
                                                                                random_state=43)

print("The validation set: ")
print(validation_set_x)

print("The test set: ")
print(test_set_x)

print("The length of validation and test set: ")
print(len(validation_set_x), len(test_set_x), len(validation_set_id), len(test_set_id))

# print validation_set_x, test_set_x, validation_set_y, test_set_y

# Now we need to convert all the *_set_y to 0 and 1 labels. All the *_set_y lists contain the actual names of all the samples.

# The samples' ids of the train_set should be reserved, so after training we can still use the ids to identify which feature sets belong to which sample.
train_set_y = generateLabels(train_set_y_id)
validation_set_y = generateLabels(validation_set_id)
test_set_y = generateLabels(test_set_id)

print("-------------------------")

print("The shape of the datasets: " + "\r\n")

print(train_set_x.shape, train_set_y.shape, validation_set_x.shape, validation_set_y.shape, test_set_x.shape,
      test_set_y.shape)

print(np.count_nonzero(train_set_y), np.count_nonzero(validation_set_y), np.count_nonzero(test_set_y))





max_len 1000
Pad sequences (samples x time)
(1229, 1000)
[[18 10 18 ...  0  0  0]
 [24 11 24 ...  0  0  0]
 [43  7 43 ...  0  0  0]
 ...
 [14  6 14 ...  0  0  0]
 [62 15 62 ...  0  0  0]
 [61 21 61 ...  0  0  0]]
The training set: 
[[15  1 15 ...  0  0  0]
 [16  5 16 ...  0  0  0]
 [ 6  8  6 ...  0  0  0]
 ...
 [ 4  0  4 ...  0  0  0]
 [58 24 58 ...  0  0  0]
 [51 17 51 ...  0  0  0]]
The length of training set: 798. The length of training set id list: 798
The validation set: 
[[32 18 32 ...  0  0  0]
 [18  5 18 ...  0  0  0]
 [11  2  3 ...  0  0  0]
 ...
 [20  2 20 ...  0  0  0]
 [48 15 48 ...  0  0  0]
 [14 10 14 ...  0  0  0]]
The test set: 
[[ 38   4  38 ...   0   0   0]
 [ 18   0  18 ...   0   0   0]
 [105  14 105 ...   0   0   0]
 ...
 [ 12   6  12 ...   0   0   0]
 [ 11   4  11 ...   0   0   0]
 [ 23   9  23 ...   0   0   0]]
The length of validation and test set: 
245 186 245 186
-------------------------
The shape of the datasets: 

(798, 1000) (798, 1) (245, 1000) (245, 1) (1

In [ ]:

print(training_list)

[[18, 10, 18, 2, 13, 0, 0, 10, 6, 1, 10, 13, 1, 1, 11, 4, 4, 6, 1, 2, 0, 2, 0], [24, 11, 24, 4, 17, 0, 0, 7, 6, 1, 8, 12, 2, 0, 10, 5, 5, 7, 4, 3, 3, 2, 3], [43, 7, 43, 12, 34, 0, 0, 5, 1161, 3, 35, 47, 10, 0, 37, 13, 13, 22, 3, 2, 2, 2, 1], [59, 14, 59, 7, 53, 0, 0, 28, 384, 3, 41, 50, 6, 0, 44, 11, 11, 11, 7, 16, 15, 1, 15], [38, 1, 38, 33, 35, 0, 0, 4, 2, 0, 4, 5, 2, 0, 3, 2, 2, 2, 1, 1, 0, 1, 0], [15, 7, 15, 4, 11, 0, 0, 9, 2, 0, 11, 12, 3, 0, 9, 2, 2, 2, 1, 1, 0, 1, 0], [31, 15, 31, 2, 24, 0, 0, 9, 19, 1, 14, 21, 1, 0, 20, 8, 8, 10, 4, 6, 3, 2, 3], [13, 1, 13, 3, 9, 0, 0, 2, 4, 1, 6, 9, 2, 0, 7, 4, 4, 4, 3, 2, 2, 2, 2], [15, 8, 15, 2, 12, 0, 0, 5, 1, 0, 12, 12, 1, 0, 11, 1, 1, 1, 1, 0, 0, 0, 0], [34, 11, 34, 7, 23, 0, 0, 5, 16, 1, 21, 27, 6, 0, 21, 7, 7, 11, 4, 6, 5, 2, 4], [13, 3, 13, 6, 7, 0, 0, 5, 2, 0, 10, 11, 5, 0, 6, 3, 3, 3, 1, 0, 0, 1, 0], [14, 0, 14, 3, 10, 0, 0, 2, 2, 0, 9, 10, 2, 0, 8, 3, 3, 3, 1, 0, 0, 2, 0], [24, 19, 24, 5, 21, 0, 0, 21, 3, 0, 19, 21, 4, 0, 17, 3, 3, 

#DEFINE NETWORK

In [ ]:
DATA_DIMENSION = 4025
model = Sequential()
#
model.add(Embedding(DATA_DIMENSION, 64))  # Layer 0: an embedding layer
model.add(Bidirectional(LSTM(64, activation='tanh', return_sequences=True))) # Layer 1: An LSTM layer (tanh)
model.add(Bidirectional(LSTM(64)))  # Layer 2: An LSTM layer
model.add(Dense(64, activation='tanh'))
model.add(Dense(32))
model.add(Dense(1, activation='sigmoid')) # Layer 3: Dense layer

print ("-------------------------")

print ("strat compiling the model...")

-------------------------
strat compiling the model...


#CONFIGURE

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Save weights of best training epoch: monitor either val_loss or val_acc
# now = datetime.now()
# callbacks_list = [
#     ModelCheckpoint(filepath=model_saved_path + now.strftime("%H:%M:%S") + '2st_64_model_{epoch:02d}_{val_loss:.3f}.h5',
#                     monitor='val_loss', verbose=2, save_best_only=True, period=1),
#     EarlyStopping(monitor='val_loss', patience=20, verbose=2, mode="min")]

callbacks_list = [
        ModelCheckpoint(filepath = model_saved_path + '2st_64_model_{epoch:02d}_{val_loss:.3f}.h5', monitor='val_loss', verbose=2, save_best_only=True, period=1),
        EarlyStopping(monitor='val_loss', patience=20, verbose=2, mode="min"),
        #TensorBoard(log_dir=log_path, batch_size = BATCH_SIZE,  write_graph=True, write_grads=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None),
        #CSVLogger(log_path + saved_model_name + '.log')
        ]

print("start training the model...")

start training the model...


#Train model

In [ ]:


# 7. Train the model. 
model.fit(train_set_x, train_set_y,
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
		   shuffle = False, # The data has already been shuffle before, so it is unnessary to shuffle it again. (And also, we need to correspond the ids to the features of the samples.)
          #validation_split=0.5,
          validation_data = (validation_set_x, validation_set_y), # Validation data is not used for training (or development of the model)
          callbacks=callbacks_list, # Get the best weights of the model and stop the first raound training.
          verbose=2)

print ("Model training completed! ")

print ("-----------------------------------------------")

print ("Start predicting....")

predicted_classes = model.predict(test_set_x, batch_size=BATCH_SIZE, verbose=2)
predicted_classes=np.argmax(predicted_classes,axis=1)
#print (predicted_classes)

test_accuracy = np.mean(np.equal(test_set_y, predicted_classes))

print ("LSTM classification result: ")

target_names = ["Non-vulnerable","Vulnerable"] #non-vulnerable->0, vulnerable->1
print (confusion_matrix(test_set_y, predicted_classes, labels=[0,1]))   
print ("\r\n")
print ("\r\n")
print (classification_report(test_set_y, predicted_classes, target_names=target_names))

print ("LSTM prediction completed.")

K.clear_session()	

print ("\r\n")
print ("--- %s seconds ---" + str(time.time() - script_start_time))

Epoch 1/50

Epoch 1: val_loss improved from inf to 0.00027, saving model to /content/drive/MyDrive/DoAnLTAT2/Model_save_path/2st_64_model_01_0.000.h5
25/25 - 11s - loss: 0.0550 - accuracy: 1.0000 - val_loss: 2.6539e-04 - val_accuracy: 1.0000 - 11s/epoch - 425ms/step
Epoch 2/50

Epoch 2: val_loss improved from 0.00027 to 0.00010, saving model to /content/drive/MyDrive/DoAnLTAT2/Model_save_path/2st_64_model_02_0.000.h5
25/25 - 3s - loss: 1.7161e-04 - accuracy: 1.0000 - val_loss: 1.0427e-04 - val_accuracy: 1.0000 - 3s/epoch - 135ms/step
Epoch 3/50

Epoch 3: val_loss improved from 0.00010 to 0.00005, saving model to /content/drive/MyDrive/DoAnLTAT2/Model_save_path/2st_64_model_03_0.000.h5
25/25 - 3s - loss: 7.3711e-05 - accuracy: 1.0000 - val_loss: 4.8652e-05 - val_accuracy: 1.0000 - 3s/epoch - 140ms/step
Epoch 4/50

Epoch 4: val_loss improved from 0.00005 to 0.00002, saving model to /content/drive/MyDrive/DoAnLTAT2/Model_save_path/2st_64_model_04_0.000.h5
25/25 - 4s - loss: 3.4794e-05 - a

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1